# Renewal benchmarking simulation

## Notes

A 1vs1 match is leads to A win, B win or a draw.
Clicks are generated by user over 30 news diplayed : 15 news from one recsys and 15 other news from another. Actually it's more complicated because we can have duplicates, but for this simulation.
For the simlatation, we only consider outcomes of a 1vs1.

Each 4 days window (`days_window`), we re-assign user to systems.
A user will click on news 2 times a day (`nb_matchs_per_day_per_user`), wich means it will click on $x$ news (among 60 news, i.e. 2 buckets of recommendations).
`nb_windows` is the number of time period we will run (with re-assignments).

The score of a system is the probability it will win against a random system.
The score is a latent variable we know for the simulation, usefull to get result of a 1vs1.
It is not known in a real competition.
For instance, for a interval of 0.6, the worst system will have a score of 0.2 and the best will have a score of 0.8. The more scores of system are close, the more we'll need matchs to get a reliable ranking of systems.

We use the [TrueSkill lib](https://trueskill.org/) (non-commercial use), equivalent to the Elo rating.
TrueSkill ([paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2007/01/NIPS2006_0688.pdf)) is a generalisation of the Elo system which handles draws, can deal with any number of competing entities and can infer individual skills from team results.

To evaluate the rating, we use the Kendall's Tau.
See the paper of this simulation for more details (CiML 2020).

## Imports

In [1]:
is_notebook = '__file__' not in locals()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# !pip install trueskill

In [3]:
from trueskill import Rating, rate_1vs1 # we asked about BETA in win_prob https://github.com/sublee/trueskill/issues/1#issuecomment-698816324
import copy
import random
# import Levenshtein as leven # !pip install python-Levenshtein
from scipy import stats

In [4]:
from systemtools.basics import *
from systemtools.printer import *
from systemtools.logger import *
from systemtools.duration import *
from systemtools.system import *
from machinelearning.iterator import *

In [5]:
from renewalsimulator.utils import *

In [6]:
from dataviztools.bokehutils import *
import bokeh
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import Grid, Legend, LegendItem
from bokeh.layouts import gridplot
from bokeh.io import export_svgs, export_png
output_notebook()

Loading BokehJS ...

## Functions for the simulation:

In [7]:
# We create a function that give the distance between 2 rankings:
def levenstein_distance(r1, r2):
    """
        This works on a ranking vector (not ranks)
    """
    if isinstance(r1, list):
        r1 = "".join([str(e) for e in r1])
    if isinstance(r2, list):
        r2 = "".join([str(e) for e in r2])
    return leven.distance(r1, r2)

In [8]:
# This is a better function we'll use:
def kendall_tau(ranks1, ranks2):
    """
        This works on a ranks vector (not ranking)
    """
    try:
        tau = stats.kendalltau(ranks1, ranks2)[0]
        assert isinstance(tau, float) and not np.isnan(tau)
        return tau
    except:
        return -1

All details of the `generate_pairwise_win_prob` function which generate a pairwise win prob matrix from a vector of win probas
is in this [StackExchange Math question](https://math.stackexchange.com/questions/3842485/how-to-generate-the-pairwise-win-probability-matrix-according-to-the-win-probabi/3842570#3842570).
The formula is not mathematically right, but is sufficient to have ground truth ranking of rec systems
and to compare it to generated rankings by the TrueSkill strategy.

$$p(\textrm{A wins against B}) = \frac{p(\textrm{A wins}) \cdot (1 - p(\textrm{B wins}))}{p(\textrm{A wins}) \cdot (1 - p(\textrm{B wins})) + (1 - p(\textrm{A wins})) \cdot p(\textrm{B wins})}$$

In [9]:
def generate_pairwise_win_prob(win_prob, float_precision=None):
    # We create the pairwise win probability (`p_win_prob`):
    p_win_prob = np.zeros((len(win_prob), len(win_prob)))
    w = win_prob
    for i in range(len(win_prob)):
        for j in range(i, len(win_prob)):
            # p_win_prob[i, j] = 1 / (1 + np.exp(w[j] - w[i])) # The Bradley-Terry-Luce model doesn't work
            # p_win_prob[i, j] = 1/(1+10**((w[j] - w[i]) / 400)) # ELO
            p_win_prob[i, j] = (w[i] * (1 - w[j])) / (w[i] * (1 - w[j]) + (1 - w[i]) * w[j])
            if float_precision is not None:
                p_win_prob[i, j] = truncateFloat(p_win_prob[i, j], float_precision)
            p_win_prob[j, i] = 1 - p_win_prob[i,j]
    return p_win_prob

In [10]:
# We define the function that will give the result of a match:
def match(i, j, p_win_prob, draw_prob_interval=None): # draw a comparision from the model
    assert i != j
    rdf = getRandomFloat()
    if draw_prob_interval is not None and abs(p_win_prob[i, j] - rdf) <= draw_prob_interval:
        return 0 # draw
    elif rdf < p_win_prob[i,j]:
        return 1 # i beats j
    else:
        return -1 # j beats i

In [11]:
def renewal_winprob(config):
    """
        This function get a config for the similation parameters and return a list of events.
        Each event is a 1vs1 match with the calculated ranking of systems.
    """
    # We create user and systems ids:
    users = list(range(config["nb_users"]))
    recsyss = list(range(config["nb_recsys"]))
    # We generate min and max from recsys_win_prob_interval:
    recsys_start_win_prob = 0.5 - config["recsys_win_prob_interval"] / 2
    recsys_end_win_prob = 0.5 + config["recsys_win_prob_interval"] / 2
    # We create the rr:
    rr = RenewalRanking(recsyss, float_precision=1)
    # We create scores of systems (`win_prob`):
    # win_prob = list(np.linspace(config["recsys_start_win_prob"], config["recsys_end_win_prob"], num=config["nb_recsys"])) # This is for linear scores
    win_prob = []
    for i in range(config["nb_recsys"]):
        rdf = getRandomFloat(recsys_start_win_prob,  recsys_end_win_prob)
        win_prob.append(truncateFloat(rdf, 3))
    win_prob = sorted(win_prob) # example : [ 0.13, 0.13, 0.16, 0.32, 0.36, 0.36, 0.38, 0.47, 0.5, 0.76 ]
    # We create the ground truth rank vector:
    ground_truth_ranks = scores2ranks(win_prob) # example : [8 8 7 6 5 5 4 3 2 1]
    p_win_prob = generate_pairwise_win_prob(win_prob)
    # We init the `infos` objects which gather general infos:
    infos = dict()
    # We simulate random matchs to re-calculate win_prob:
    victories = [0] * len(recsyss)
    defeats = [0] * len(recsyss)
    for i in range(1000 * len(recsyss)):
        a, b = random.sample(range(len(recsyss)), 2)
        result = match(a, b, p_win_prob, draw_prob_interval=config["draw_prob_interval"])
        if result != 0:
            if result == 1:
                victories[a] += 1
                defeats[b] += 1
            else:
                victories[b] += 1
                defeats[a] += 1
    predicted_win_prob = []
    for i in range(len(recsyss)):
        current = victories[i] / (victories[i] + defeats[i])
        predicted_win_prob.append(current)
    infos["predicted_win_prob"] = predicted_win_prob
    infos["predicted_win_prob_interval"] = max(predicted_win_prob) - min(predicted_win_prob)
    return infos

In [12]:
def renewal_simulate(config):
    """
        This function get a config for the similation parameters and return a list of events.
        Each event is a 1vs1 match with the calculated ranking of systems.
    """
    # We create user and systems ids:
    users = list(range(config["nb_users"]))
    recsyss = list(range(config["nb_recsys"]))
    # We generate min and max from recsys_win_prob_interval:
    recsys_start_win_prob = 0.5 - config["recsys_win_prob_interval"] / 2
    recsys_end_win_prob = 0.5 + config["recsys_win_prob_interval"] / 2
    # We create the rr:
    rr = RenewalRanking(recsyss, float_precision=1)
    # We create scores of systems (`win_prob`):
    # win_prob = list(np.linspace(config["recsys_start_win_prob"], config["recsys_end_win_prob"], num=config["nb_recsys"])) # This is for linear scores
    win_prob = []
    for i in range(config["nb_recsys"]):
        rdf = getRandomFloat(recsys_start_win_prob,  recsys_end_win_prob)
        win_prob.append(truncateFloat(rdf, 3))
    win_prob = sorted(win_prob) # example : [ 0.13, 0.13, 0.16, 0.32, 0.36, 0.36, 0.38, 0.47, 0.5, 0.76 ]
    # We create the ground truth rank vector:
    ground_truth_ranks = scores2ranks(win_prob) # example : [8 8 7 6 5 5 4 3 2 1]
    p_win_prob = generate_pairwise_win_prob(win_prob)
    # We init the event variable (which will store each match with ranking, winner, etc.):
    events = []
    # Init of vars to early stop:
    best_score = -2
    best_window_score = -2
    unchanged_window_score_count = 0
    # Infinite competition until we early stop:
    for window in range(config["nb_windows"]):
        # First, we assign 2 systems to each user, so 1 system have a subset of all users (it's the "re-assignment" phase):
        assignments = renewal_assignments(users, recsyss)
        # Here we create all matchs:
        for day in range(config["days_window"]):
            for user in range(config["nb_users"]):
                # Here the "real" usage of the app per user per day is between 0 and 2*n
                # We don't use this because we need consistent day windows:
                # current_nb_matchs = random.choice(range(config["max_nb_matchs_per_day_per_user"] + 1))
                for _ in range(config["nb_matchs_per_day_per_user"]):
                    # Contenders are those assigned to the current user:
                    contenders = list(assignments[user])
                    # We get the result of the current match:
                    result = match(contenders[0], contenders[1], p_win_prob, draw_prob_interval=config["draw_prob_interval"])
                    # And we add the result to the rating strategy (`rr`):
                    winner = None
                    if result == 1:
                        winner = contenders[0]
                    elif result == -1:
                        winner = contenders[1]
                    rr.match(contenders[0], contenders[1], winner=winner)
                    # We get ranks:
                    ranks = rr.get_ranks()
                    # We compute the score between -1 and 1:
                    kendalltau = kendall_tau(ranks, ground_truth_ranks)
                    # We create the current "event" which is a match result with a new ranking of all systems:
                    event = \
                    {
                        "match": len(events), "window": window,
                        # "user": user,
                        # "a": contenders[0], "a_win_prob": win_prob[contenders[0]],
                        # "a_win_prob_against_b": p_win_prob[contenders[0], contenders[1]],
                        # "b": contenders[1], "b_win_prob": win_prob[contenders[1]],
                        # "winner": winner, "ranks": rr.get_ranks(),
                        # "ranking_stats": rr.get_ranking_stats(),
                        "kendall_tau": kendalltau,
                        "day": len(events) / (config["nb_users"] * config["nb_matchs_per_day_per_user"]),
                    }
                    # Finally we save the current event and the score if it's a better one:
                    events.append(event)
                    if event["kendall_tau"] > best_score:
                        best_score = event["kendall_tau"]
        # We early stop in case the best score is unchanged for a certain amount of windows:
        if best_score > best_window_score:
            best_window_score = best_score
            unchanged_window_score_count = 0
        else:
            unchanged_window_score_count += 1
        if config["early_stop_windows"] is not None and unchanged_window_score_count > config["early_stop_windows"]:
            break
    return events

In [13]:
def mean_events_and_infos(eventss, config):
    # Deep copies:
    # eventss = copy.deepcopy(eventss)
    # infoss = copy.deepcopy(infoss)
    # We create infos:
    mean_infos = dict()
    # First we cut longer events:
    min_length = min([len(e) for e in eventss])
    max_length = max([len(e) for e in eventss])
    for i in range(len(eventss)):
        eventss[i] = eventss[i][:min_length]
    # We mean all:
    mean_events = []
    for i in range(min_length):
        event = dict()
        event["kendall_tau"] = 0
        all_taus = []
        for events in eventss:
            all_taus.append(events[i]["kendall_tau"])
        event["kendall_tau"] = sum(all_taus) / len(all_taus)
        event["kendall_tau_std"] = np.std(all_taus)
        event["window"] = eventss[0][i]["window"]
        event["match"] = eventss[0][i]["match"]
        event["day"] = eventss[0][i]["day"]
        mean_events.append(event)
    # We enrich infos according to the mean:
    best_score = -1
    for event in mean_events:
        if event["kendall_tau"] > best_score:
            best_score = event["kendall_tau"]
    minor_best_score = best_score - config["best_score_interval"]
    minor_best_score_days = 0
    minor_best_score_windows = 0
    minor_best_score_matchs = 0
    for i in range(len(mean_events)):
        if mean_events[i]["kendall_tau"] >= minor_best_score:
            minor_best_score_days = mean_events[i]["day"]
            minor_best_score_windows = mean_events[i]["window"]
            minor_best_score_matchs = mean_events[i]["match"]
            break
    mean_infos["minor_best_score_days"] = minor_best_score_days
    mean_infos["minor_best_score_windows"] = minor_best_score_windows
    mean_infos["minor_best_score_matchs"] = minor_best_score_matchs
    mean_infos["minor_best_score"] = minor_best_score
    mean_infos["best_score"] = best_score
    # We return all:
    return (mean_events, mean_infos)

In [14]:
def cut_eventss(mean_events, mean_infos):
    cut_window = math.ceil(mean_infos["minor_best_score_windows"] + 1)
    if cut_window <= 1:
        cut_window = 2
    if cut_window > 20:
        cut_window += 5
    elif cut_window > 5:
        cut_window += 2
    new_mean_events = []
    for event in mean_events:
        if event["window"] == cut_window:
            break
        else:
            new_mean_events.append(event)
    return new_mean_events

In [15]:
def display_infos\
(
    mean_events,
    mean_infos,
    config,
    width=200,
    height=200,
    title=None,
    logger=None,
    path=None,
):
    # TODO Count the number of 1vs1 which didn't appear before re-assignments to see gaps
    log("Number of simulations: " + str(config["nb_simulations"]), logger=logger)
    log("Result: " + str(truncateFloat(mean_infos["best_score"], 2)) + " (" + str(truncateFloat(mean_infos["minor_best_score_days"], 2)) + " days, " + str(mean_infos["minor_best_score_matchs"]) + " matchs)", logger=logger)
    events = mean_events
    # x_axis_label, y_axis_label = "Day", "Kendall's τ"
    x_axis_label, y_axis_label = "Day", "Ranking reliability"
    # x_axis_label, y_axis_label = None, None
    x_axis, y_axis = "day", "kendall_tau"
    TOOLTIPS = None # [("window", "@window")]
    data = copy.deepcopy(events)
    data = listOfDictToDictOfList(data)
    p = figure\
    (
        title=title, x_axis_label=x_axis_label, y_axis_label=y_axis_label,
        tooltips=TOOLTIPS,
        width=width, height=height,
    )
    p.title.align = 'center'
    # We get the worst score:
    worst_score = 1
    for e in events:
        if e["kendall_tau"] < worst_score:
            worst_score = e["kendall_tau"]
    # We show first std curve:
    x_data = []
    top_std_curve = []
    bottom_std_curve = []
    for event in events:
        std95 = event["kendall_tau_std"] * 2
        top = event[y_axis] + std95
        bottom = event[y_axis] - std95
        if bottom >= worst_score:
            x_data.append(event[x_axis])
            top_std_curve.append(top)
            bottom_std_curve.append(bottom)
    p.line(x_data, top_std_curve, width=1, color=colors[4], line_alpha=0.8)
    p.line(x_data, bottom_std_curve, width=1, color=colors[4], line_alpha=0.8)
    # Re-assignment lines:
    p.line(x_axis, y_axis, width=2, source=ColumnDataSource(data), color=colors[1])
    # Scores:
    for i in range(1, len(events)):
        if events[i]['window'] != events[i - 1]['window']:
            p.line([events[i][x_axis], events[i][x_axis]], [worst_score, 1], width=1, line_dash="dashed", color=colors[3])
    show(p)
    try:
        export_png(p, filename=title + ".png")
    except: pass

In [16]:
def simulate\
(
    name,
    nb_simulations,
    nb_users,
    nb_recsys,
    early_stop_windows,
    recsys_win_prob_interval,
    logger=None,
    stop_at_pwinprob=False,
):
    config = \
    {
        # -------------------------------------- #
        "name": name,
        "nb_simulations": nb_simulations,
        "nb_users": nb_users,
        "nb_recsys": nb_recsys,
        "early_stop_windows": early_stop_windows,
        "recsys_win_prob_interval": recsys_win_prob_interval,
        # -------------------------------------- #
        "days_window": 4,
        "nb_matchs_per_day_per_user": 2,
        "draw_prob_interval": 0.2,
        "best_score_interval": 0.05,
        # -------------------------------------- #
    }
    config["nb_windows"] = int(30 / config["days_window"] * 8) # 4 months
    bp(config, 5, logger)
    infoss = []
    nb_simulations = 48 * 4
    def gen_funct(configs, logger=None, verbose=True):
        for config in configs:
            yield renewal_winprob(config)
    configs = chunks([copy.deepcopy(config) for i in range(nb_simulations)], 1)
    mli = MLIterator(configs, gen_funct, parallelProcesses=cpuCount(), maxParallelProcesses=cpuCount(), logger=logger)
    for infos in mli:
        infoss.append(infos)
    log("Average predicted_win_prob_interval: " + str(np.mean([e["predicted_win_prob_interval"] for e in infoss])), logger=logger)
    if stop_at_pwinprob:
        return
    eventss = []
    nb_simulations = config["nb_simulations"]
    def gen_funct(configs, logger=None, verbose=True):
        for config in configs:
            yield renewal_simulate(config)
    configs = chunks([copy.deepcopy(config) for i in range(nb_simulations)], 1)
    mli = MLIterator(configs, gen_funct, parallelProcesses=cpuCount(), maxParallelProcesses=cpuCount(), logger=logger)
    for events in mli:
        eventss.append(events)
    log(str(len(eventss)) + " simulations generated.", logger)
    (mean_events, mean_infos) = mean_events_and_infos(eventss, config)
    mean_events = cut_eventss(mean_events, mean_infos)
    # bp(mean_infos)
    log(str(len(mean_events)) + " events.", logger=logger)
    display_infos\
    (
        mean_events,
        mean_infos,
        config,
        width=200,
        height=200,
        title=config["name"],
        logger=logger,
        path=logRoot,
    )

## Init

In [17]:
logRoot = homeDir() + "/renewsim-logs"
mkdir(logRoot)
loggerPath = logRoot + "/" + getDateSec() + ".log"
print(loggerPath)

/home/hayj/renewsim-logs/2020.10.01-21.11.06.log


In [18]:
logger = Logger(loggerPath)

In [19]:
# https://learnui.design/tools/data-color-picker.html#palette
palettes = \
[
    ['#488f31', '#a7c162', '#fff59f', '#f49e5c', '#de425b'],
    ['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ffa600'],
    ['#009b95', '#007acc', '#bc5090', '#aaaaaa', '#ff8f8f', '#ef9a32'],
    ['#a31430', '#0071bd', '#02bebf', '#df7d00', '#017f01'],
]
colors = palettes[2]

## Simulation execution

In [ ]:
simulate\
(
    name="Simulation 1",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.42,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 2",
    nb_simulations=400,
    nb_users=1000,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.42,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 3",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=100,
    early_stop_windows=30,
    recsys_win_prob_interval=0.37,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 4",
    nb_simulations=400,
    nb_users=1000,
    nb_recsys=100,
    early_stop_windows=10,
    recsys_win_prob_interval=0.37,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 5",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.27,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 6",
    nb_simulations=400,
    nb_users=1000,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.27,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 7",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=100,
    early_stop_windows=30,
    recsys_win_prob_interval=0.23,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 8",
    nb_simulations=400,
    nb_users=1000,
    nb_recsys=100,
    early_stop_windows=10,
    recsys_win_prob_interval=0.23,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 9",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.13,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 10",
    nb_simulations=400,
    nb_users=1000,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.13,
    logger=logger,
    stop_at_pwinprob=False,
)

In [ ]:
simulate\
(
    name="Simulation 11",
    nb_simulations=400,
    nb_users=100,
    nb_recsys=100,
    early_stop_windows=30,
    recsys_win_prob_interval=0.1,
    logger=logger,
    stop_at_pwinprob=False,
)

## Coding week with students

In [20]:
simulate\
(
    name="1000 users, 100 sys.",
    nb_simulations=48,
    nb_users=5 * 10,
    nb_recsys=10,
    early_stop_windows=10,
    recsys_win_prob_interval=0.3,
    logger=logger,
    stop_at_pwinprob=False,
)

{ 'best_score_interval': 0.05, 'days_window': 4, 'draw_prob_interval': 0.2, 'early_stop_windows': 10, 'name': 1000 users and 100 systems, 'nb_matchs_per_day_per_user': 2, 'nb_recsys': 10, 'nb_simulations': 48, 'nb_users': 50, 'nb_windows': 60, 'recsys_win_prob_interval': 0.3 }
With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
192 containers to process.
  0% [                    ]
  9% [=                   ] (6.373s left)
 19% [===                 ] (5.106s left)
 29% [=====               ] (4.428s left)
 39% [=======             ] (3.785s left)
 49% [=========           ] (3.195s left)
 59% [===========         ] (2.606s left)
 69% [=============       ] (1.978s left)
 79% [===============     ] (1.3s left)
 89% [=================   ] (0.675s left)
 98% [=================== ] (0.063s left)
100% [====================] (total duration: 6.05s, mean duration: 0.031s)
Average predicted_win_prob_interv

## Tests

In [ ]:
if False:
    # We print stats:
    for i in range(len(recsyss)):
        print("The prob the system " + str(i) + " wins against a random system is " + str(win_prob[i]))
    for i in range(len(recsyss)):
        for j in range(len(recsyss)):
            if i != j:
                print(str(i) + " wins against " + str(j) + " with a prob of " + str(p_win_prob[i, j]))

In [ ]:
if False:
    # We test the match function:
    for _ in range(10):
        contenders = random.sample(recsyss, 2)
        i, j = contenders[0], contenders[1]
        print(str(i) + " wins against " + str(j) + " with a prob of " + str(p_win_prob[i, j]))
        match_result = match(i, j)
        if match_result == 0:
            print("Draw")
        elif match_result == 1:
            print(str(i) + " win")
        else:
            print(str(j) + " win")
        print()